### Dataset 구축하기 

In [ ]:
# 키 불러오기 
from google.colab import files
uploaded= files.upload()
filename= list(uploaded.keys())[0]
filename

Saving key.txt to key.txt


'key.txt'

In [ ]:
# api 키 확인 
with open(filename) as f:
  api_key = f.read()
len(api_key)

88

In [ ]:
# 필요한 라이브러리 import 
import requests
from urllib.parse import quote
import requests, bs4
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote

각 증상별 적합한 약품 추천 Dataset 구축(1) - 모든 데이터 추출 

In [ ]:
sym  = ['두통', '염증', '상처', '과식', '구토', '복통','발열','가려움', '안구건조', '습진', '생리통',
        '외상','어지러움', '치통', '통증', '진통', '설사', '기침','오한', '근육통', '염좌',
        '타박상', '신경통', '요통', '어깨결림', '소염', '체함', '화상', '기침',
        '가래', '속쓰림', '코감기', '변비', '피로', '인후통', '물집', '탈모', '비타민']

In [ ]:
# 여러개 증상의 api 호출하는 함수 
def call_sym(sym):
  med_total = []
  med_url = 'http://apis.data.go.kr/1471000/DrbEasyDrugInfoService/getDrbEasyDrugList'
  My_API_Key = api_key
  # get 방식으로 쿼리를 분리하기 위해 '?'를 넣은 것
  for i in range(len(sym)):
    queryParams = '?' + urlencode(                                  
        {
            quote_plus('ServiceKey') : My_API_Key,                    # 필수 : 서비스키 (본인의 서비스키) 
            #quote_plus('itemName') : '한미아스피린장용정100밀리그램',          
            quote_plus('efcyQesitm') : sym[i]                          # 찾는 parameter   
        }
    )
    response = requests.get(med_url + queryParams).text.encode('utf-8')
    xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')
    med_list = xmlobj.find_all('itemName')

    name_list = []
    for n in med_list:
      name = n.text
      name_list.append(name)
    
    med_total.append(name_list)
  return med_total

In [ ]:
# 딕셔너리 데이터셋 구축 
dictionary = dict(zip(sym, call_sym(sym)))
dictionary

{'가래': ['자모',
  '코푸시럽에스',
  '판콜에이내복액',
  '리나치올시럽(카르보시스테인)(수출명:HyundiolSyrup)',
  '리나치올캡슐375밀리그램(카르보시스테인)(수출명:HyundiolCapsule375Mg)',
  '뮤코론캡슐375밀리그램(카르보시스테인)',
  '리나치올시럽5%(카르보시스테인)(수출명:HyundiolSyrup5%)',
  '리나치올캡슐500밀리그람(L-카르보시스테인)(수출명:HyundiolCapsule500Mg)',
  '크린조(염화나트륨)',
  '판콜에스내복액'],
 '가려움': ['페니라민정(클로르페니라민말레산염)',
  '에어신신파스',
  '쎄레스톤지크림',
  '유락신연고(크로타미톤)',
  '제이알히드로코르티손연고',
  '삼아리도멕스크림(프레드니솔론발레로아세테이트)',
  '신신물파스에스(수출명:HEATWAVE,ARTHROHONLINIMENT,PANOPAS,Y&JWATER)',
  '삼아리도멕스로션(프레드니솔론발레로아세테이트)',
  '청나점안액',
  '보송크림(프레드니솔론발레로아세테이트)'],
 '과식': ['활명수',
  '보화소합원(대환,소환)',
  '베스타제정',
  '베스타제당의정',
  '건위황정',
  '잘젠정',
  '제스판골드정',
  '베로타정',
  '다이제틴정',
  '리포유정'],
 '구토': ['활명수',
  '아네모정',
  '겔포스현탁액(인산알루미늄겔)',
  '게루삼정',
  '보화소합원(대환,소환)',
  '보나링에이정(디멘히드리네이트)',
  '건위황정',
  '제스판골드정',
  '포리부틴정(트리메부틴말레산염)',
  '스파부틴정100밀리그램(트리메부틴말레산염)'],
 '근육통': ['에어신신파스',
  '판콜에이내복액',
  '아로나민골드정',
  '뇌선',
  '부루펜정200밀리그램(이부프로펜)',
  '게보린정(수출명:돌로린정)',
  '바로론정(클로닉신리시네이트)',
  '녹십자제놀카타플라스마',
  '부루펜정400밀리그램(이부프로펜)',
  '녹십자제놀마

각 증상별 적합한 약품 추천 Dataset 구축(2) - 3개만 추출, 괄호 삭제

In [ ]:
# 괄호 삭제하기 
txt = '페니라민정(클로르페니라민말레산염)'
txt = txt.split('(')[0]
txt

'페니라민정'

In [ ]:
# 3개만 호출 
def call_sym_short(sym):
  med_total = []
  med_url = 'http://apis.data.go.kr/1471000/DrbEasyDrugInfoService/getDrbEasyDrugList'
  My_API_Key = api_key
  # get 방식으로 쿼리를 분리하기 위해 '?'를 넣은 것
  for i in range(len(sym)):
    queryParams = '?' + urlencode(                                  
        {
            quote_plus('ServiceKey') : My_API_Key,                    # 필수 : 서비스키 (본인의 서비스키) 
            #quote_plus('itemName') : '한미아스피린장용정100밀리그램',          
            quote_plus('efcyQesitm') : sym[i]                          # 찾는 parameter   
        }
    )
    response = requests.get(med_url + queryParams).text.encode('utf-8')
    xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')
    med_list = xmlobj.find_all('itemName')

    name_list = []
    for n in med_list[:3]:
      raw_name = n.text
      name = raw_name.split('(')[0]
      name_list.append(name)
    
    med_total.append(name_list)
  return med_total

In [ ]:
dict_short = dict(zip(sym, call_sym_short(sym)))
dict_short

{'가래': ['자모', '코푸시럽에스', '판콜에이내복액'],
 '가려움': ['페니라민정', '에어신신파스', '쎄레스톤지크림'],
 '과식': ['활명수', '보화소합원', '베스타제정'],
 '구토': ['활명수', '아네모정', '겔포스현탁액'],
 '근육통': ['에어신신파스', '판콜에이내복액', '아로나민골드정'],
 '기침': ['자모', '페니라민정', '코푸시럽에스'],
 '두통': ['판콜에이내복액', '뇌선', '낙센정'],
 '물집': ['메노바크림', '고려아시클로버크림', '에크로바크림'],
 '발열': ['판콜에이내복액', '제로미아액', '뇌선'],
 '변비': ['성광관장약', '마그밀정', '신일비사코딜정'],
 '복통': ['포리부틴정', '스파부틴정100밀리그램', '타라부틴정'],
 '비타민': ['삐콤정', '아로나민골드정', '복합우루사연질캡슐'],
 '상처': ['에어신신파스', '성광멸균바세린거즈', '미보'],
 '생리통': ['뇌선', '게보린정', '뇌신에이산'],
 '설사': ['후라베린큐시럽', '로프민캡슐', '갈타제산'],
 '소염': ['에어신신파스', '녹십자제놀카타플라스마', '녹십자제놀마일드핫트카타플라스마'],
 '속쓰림': ['아네모정', '겔포스현탁액', '게루삼정'],
 '습진': ['페니라민정', '쎄레스톤지크림', '태극아즈렌에스연고'],
 '신경통': ['에어신신파스', '아로나민골드정', '뇌선'],
 '안구건조': ['리프레쉬플러스점안액0.5%', '인프레쉬플러스점안액', '리프레쉬티어즈점안액0.5%'],
 '어깨결림': ['에어신신파스', '아로나민골드정', '녹십자제놀카타플라스마'],
 '어지러움': ['보나링에이정', '보미롱산', '메카인정'],
 '염좌': ['뇌선', '부루펜정200밀리그램', '게보린정'],
 '염증': ['쎄레스톤지크림', '키모랄에스정', '기모타부정'],
 '오한': ['판콜에이내복액', '뇌선', '게보린정'],
 '외상': ['키모랄에스정',

* 딕셔너리를 파일로 저장

In [ ]:
import pickle

with open('dict_short.pickle', 'wb') as fw:
  pickle.dump(dict_short, fw)

In [ ]:
with open('dict_short.pickle','rb') as fr:
  user_loaded = pickle.load(fr)

In [ ]:
user_loaded

{'가래': ['자모', '코푸시럽에스', '판콜에이내복액'],
 '가려움': ['페니라민정', '에어신신파스', '쎄레스톤지크림'],
 '과식': ['활명수', '보화소합원', '베스타제정'],
 '구토': ['활명수', '아네모정', '겔포스현탁액'],
 '근육통': ['에어신신파스', '판콜에이내복액', '아로나민골드정'],
 '기침': ['자모', '페니라민정', '코푸시럽에스'],
 '두통': ['판콜에이내복액', '뇌선', '낙센정'],
 '물집': ['메노바크림', '고려아시클로버크림', '에크로바크림'],
 '발열': ['판콜에이내복액', '제로미아액', '뇌선'],
 '변비': ['성광관장약', '마그밀정', '신일비사코딜정'],
 '복통': ['포리부틴정', '스파부틴정100밀리그램', '타라부틴정'],
 '비타민': ['삐콤정', '아로나민골드정', '복합우루사연질캡슐'],
 '상처': ['에어신신파스', '성광멸균바세린거즈', '미보'],
 '생리통': ['뇌선', '게보린정', '뇌신에이산'],
 '설사': ['후라베린큐시럽', '로프민캡슐', '갈타제산'],
 '소염': ['에어신신파스', '녹십자제놀카타플라스마', '녹십자제놀마일드핫트카타플라스마'],
 '속쓰림': ['아네모정', '겔포스현탁액', '게루삼정'],
 '습진': ['페니라민정', '쎄레스톤지크림', '태극아즈렌에스연고'],
 '신경통': ['에어신신파스', '아로나민골드정', '뇌선'],
 '안구건조': ['리프레쉬플러스점안액0.5%', '인프레쉬플러스점안액', '리프레쉬티어즈점안액0.5%'],
 '어깨결림': ['에어신신파스', '아로나민골드정', '녹십자제놀카타플라스마'],
 '어지러움': ['보나링에이정', '보미롱산', '메카인정'],
 '염좌': ['뇌선', '부루펜정200밀리그램', '게보린정'],
 '염증': ['쎄레스톤지크림', '키모랄에스정', '기모타부정'],
 '오한': ['판콜에이내복액', '뇌선', '게보린정'],
 '외상': ['키모랄에스정',